In [51]:
import os
import urllib
import re
from urllib import request
import json
import requests
import xml.etree.ElementTree as ET
import pickle

In [52]:
####base url and apikey
#### it requires an apiKey from Elsevier
#doi = '10.1016/j.enggeo.2012.07.017'
apiKey = 'c4a233b62aa718d780cc6f2a35d2cc14'
base = 'https://api.elsevier.com/content/article/doi/'

In [53]:
def retrieveTextPaper(url, apikey, doi):
    params = {
        'APIKey': apikey,
        'view': 'FULL',
        'httpAccept': 'text/xml'  #### application/json    text/xml
        }
    post_args = urllib.parse.urlencode(params)

    test = request.urlopen(url+doi+'?'+post_args)
    html = test.read()
#     save()

    html = ET.fromstring(html)
    paragraphs = []

    for n in html:
        if 'originalText' in n.tag:
            OT = n
            break

    for n in OT:
        if 'doc' in n.tag:
            doc = n
            break

    for n in doc:
        if 'serial' in n.tag:
            serial = n
            break

    for n in serial:
        if 'article' in n.tag:
            article = n
            break

    for n in article:
        if 'body' in n.tag:
            body = n
            break

    for n in body:
        if 'sections' in n.tag:
            sections = n
            break

    for n in sections:
        for m in n:
            if 'para' in m.tag:
                # TODO: Remove refs, tables, and eq.
                paragraphs.append(''.join(m.itertext()))

    text1 = ''.join(paragraphs)
    return text1

In [54]:
####finding dois of papers to use

##base url
base2 = 'https://api.elsevier.com/content/search/scidir'

def getDOIs(url, query, apikey, subscribed='true', counts = 200, year = None):
    dois = []
    if year is None:
        for n in range(1, 5801,200):
            try:
                if n == 1:
                    param2 = {
                        'query': '{} AND pub-date IS {}'.format(query, year),
                        'APIKey': apikey,
                        'subscribed': subscribed,
                        'count': counts
                    }
                else:
                    param2 = {
                        'query': '{} AND pub-date IS {}'.format(query, year),
                        'APIKey': apikey,
                        'subscribed': subscribed,
                        'count': counts,
                        'start': str(n)
                    }
                post_args2 = urllib.parse.urlencode(param2)

                newTest = request.urlopen(base2+'?'+post_args2)
                html = newTest.read()
                for z in json.loads(html.decode("utf-8"))['search-results']['entry']:
                    dois.append(z['dc:identifier'][4:])
            except:
                continue


    else:
        year = year
        param2 = {
            'query': '{} AND pub-date IS {}'.format(query, year),
            'APIKey': apikey,
            'subscribed': subscribed,
            'count': counts
            }
        post_args2 = urllib.parse.urlencode(param2)
        newTest = request.urlopen(base2+'?'+post_args2)
        html = newTest.read()
        la = json.loads(html.decode("utf-8"))['search-results']
        number = int(la['opensearch:totalResults'])
        print(year, number)
        for n in range(1,number,200):
            if n == 1:
                param2 = {
                    'query': '{} AND pub-date IS {}'.format(query, year),
                    'APIKey': apikey,
                    'subscribed': subscribed,
                    'count': counts
                    }
            else:
                param2 = {
                    'query': '{} AND pub-date IS {}'.format(query, year),
                    'APIKey': apikey,
                    'subscribed': subscribed,
                    'count': counts,
                    'start': n
                    }
            post_args2 = urllib.parse.urlencode(param2)
            newTest = request.urlopen(base2+'?'+post_args2)
            html = newTest.read()
            for z in json.loads(html.decode("utf-8"))['search-results']['entry']:
                dois.append(z['dc:identifier'][4:])

    return dois

In [56]:
####to get all DOIs from 2000, to 2018 using the keyword lithology
allDOI = []
for n in range(2003, 2006, 1):
    DOIs = getDOIs(base2, 'geology', apiKey, year = n)
    allDOI.append(DOIs)

allDOI = [item for sublist in allDOI for item in sublist]
len(allDOI)

2003 4124


HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
##saving DOI list as picke object
with open('allDOI.pickle', 'wb') as f:
    pickle.dump(allDOI, f)
##importing DOI list saved
with open('allDOI.pickle', 'rb') as f:
    tmp = pickle.load(f)

In [48]:
##to download all papers from DOIs list as txt files into the paper folder
##The number of papers is over 30,000, so it takes a while to finish!

for n in allDOI:
    try:

        doi = n
        content = retrieveTextPaper(url = base, apikey = apiKey, doi=doi)
        if len(content) != 0:
            with open('papers\{}.txt'.format(doi.replace('/', '-')), 'w') as files:
                files.write(str(content.encode("utf-8")))
                files.close
    except:
        continue

In [32]:


params = {
        'APIKey': apiKey,
        'view': 'FULL',
        'httpAccept': 'text/xml'  #### application/json    text/xml
        }
post_args = urllib.parse.urlencode(params)

test = request.urlopen(base+allDOI[2]+'?'+post_args)
html = test.read()
#     save()

html = ET.fromstring(html)
# paragraphs = []
html
for n in html:
    if 'originalText' in n.tag:
        OT = n
        break

for n in OT:
    if 'doc' in n.tag:
        doc = n
        break

for n in doc:
    if 'serial' in n.tag:
        serial = n
        break

for n in serial:
    if 'article' in n.tag:
        article = n
        break

for n in article:
    if 'body' in n.tag:
        body = n
        break

for n in body:
    if 'sections' in n.tag:
        sections = n
        break

for n in sections:
    for m in n:
        if 'para' in m.tag:
            # TODO: Remove refs, tables, and eq.
            paragraphs.append(''.join(m.itertext()))
# print(paragraphs)
text1 = ''.join(paragraphs)
print(text1)
#     return text1

The formation of lateritic Au deposits and anomalies in the regolith depends on the mobilization, transport and precipitation of Au in the weathering environment (Wilson, 1984). Sequential extractions have been used to study the association of Au and other trace elements with principle regolith fractions (Boyle, 1979; Lintern, 1989; Gray and Lintern, 1998; Xueqiu, 1998), and to infer element mobility. Other studies have shown that Au is mobile in the Australian regolith in and around mineralized zones (Mann, 1984; Gray and Lintern, 1998; Reith et al., 2005). For instance, at the Tomakin Park Gold Mine in south eastern New South Wales approximately 50wt% of the total Au in the Ah-horizon overlying the mineralized zone was associated with the water-, ammonium-acetate-, sodium pyrophosphate- and hydroxylamine-hydrochloride-soluble fractions; in contrast, in the unweathered quartz-vein material more than 95wt% of the Au was extractable only with concentrated aqua regia and appeared to be s